In [ ]:
import json
import pandas as pd
from pathlib import Path
from datetime import datetime


In [ ]:
# Defining paths
GROUP_DATA_DIR = Path("/Users/harshul/projects/kgwebsite/group-data")

MEMBERS_DIR_PATH = GROUP_DATA_DIR / 'members/'
WEBSITE_DATA_PATH = GROUP_DATA_DIR / 'website_data/'
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / 'content'


In [ ]:
def parse_education_experience_df(member_dir):
    member_json_dir = member_dir / 'jsons'
    education_experience_list = []

    if (member_experiences_dir := (member_json_dir / 'experiences.json')).exists():
        with open(member_experiences_dir, 'r') as f:
            education_experience_list += json.load(f)
    else:
        print(member_experiences_dir)
    
    if (member_education_dir := (member_json_dir / 'education.json')).exists():
        with open(member_education_dir, 'r') as f:
            education_experience_list += json.load(f)
    
    return pd.DataFrame(education_experience_list)

In [ ]:
for member_dir in MEMBERS_DIR_PATH.glob('*'):
    if not (member_info_fname := member_dir / 'info.json').exists():
      continue
    else:
        member_info = json.load(open(member_info_fname, 'r'))
    print(member_dir, type(member_dir))

    education_experience_df = parse_education_experience_df(member_dir)



In [ ]:
member_dir = Path("/Users/harshul/projects/kgwebsite/group-data/members/isaac_smith")

parse_education_experience_df(member_dir)

In [ ]:
def extract_member_academic_role(education_experience_df):
    current_academic_role = ""
    is_current_member = False
    
    if 'end_date' not in education_experience_df.columns:
        education_experience_df['end_date'] = None
    if 'group' not in education_experience_df.columns:
        education_experience_df['group'] = None
    if 'institution' not in education_experience_df.columns:
        education_experience_df['institution'] = None    
    # Filter based on 'institution' or 'group'
    condition_institution = (education_experience_df['institution'] == 'Michigan State University')
    condition_group = education_experience_df['group'].isin(['kerzendorf', 'DTI', 'ICER'])
    
    filtered_e_e_df = education_experience_df[condition_institution | condition_group]
    
    if len(filtered_e_e_df) == 0: 
       pass
    
    elif len(filtered_e_e_df) >= 1:
        role = filtered_e_e_df.iloc[0].get('role', None)  
        degree = filtered_e_e_df.iloc[0].get('degree', None)  
        
        if role in ['Assistant Professor', 'Professor']:
            current_academic_role = 'Professor'
        if role == "Professorial Assistant":
            current_academic_role = 'Professorial Assistant'
        elif role in ['Visiting Researcher', 'Postdoctoral Researcher']:
            current_academic_role = 'Postdoctoral Researcher' 
        elif degree in ['Masters', 'PhD']:
            current_academic_role = 'Graduate Student' 
        elif degree == 'Bachelors':
            current_academic_role = 'Graduate Student' if filtered_e_e_df.iloc[0].get('end_date') is None else 'Undergraduate Student'
        
    has_end_date = any(not pd.isna(date) for date in filtered_e_e_df['end_date'])
    is_current_member = not has_end_date

    return current_academic_role, is_current_member

In [ ]:

# for creating two lists

current_people_page_list = []
alumni_people_page_list = []


for member_dir in MEMBERS_DIR_PATH.glob('*'):
    print(member_dir)
    if not (member_info_fname := member_dir / 'info.json').exists():
        continue
    else:
        member_info = json.load(open(member_info_fname, 'r'))
    education_experience_df = parse_education_experience_df(member_dir)
    current_academic_role, is_current_member = extract_member_academic_role(education_experience_df)

    
    first_name = member_info['first_name']
    print(first_name)
    last_name = member_info['last_name']
    nickname = member_info.get('nick_name', None)
    id = member_info['id']
    image_path = member_info['image_path']  
    cover_image_path = member_info['cover_image_path']   

    name = f'{nickname if nickname else first_name} {last_name}'

    if is_current_member:
        current_people_page_list.append({"Name": name, "academic_role": current_academic_role, "ID": id})
    else:
        alumni_people_page_list.append({"Name": name, "academic_role": current_academic_role, "ID": id})




In [ ]:
current_people_page_list

In [ ]:
alumni_people_page_list

In [ ]:
member_dir

In [ ]:
filtered_e_e_df = filtered_e_e_df[filtered_e_e_df['institution'] == 'Michigan State University']

In [ ]:
filtered_e_e_df